In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import os.path
import altair as alt
from pandas.io.json import json_normalize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import json
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [2]:
# df = """
# select merchant_id, repeat_loan, topup, COALESCE(Channel, 'Offline') AS Channel,
# COALESCE(running_bureau_loans,0) running_bureau_loans
# from  (select distinct lrv.merchant_id, repeat_loan, risk_segment,Channel,running_bureau_loans,
#        case when upper(risk_segment)='TOPUP' then 1 else 0 end as topup
#        from `easy_loan.lending_risk_variables` lrv
#        left join bharatpe-analytics-prod.bharatpe_ml_data.merchant_channel mc
#        on lrv.merchant_id = mc.merchant_id
#        left join `bharatpe-analytics-prod.bharatpe_ml_data.bureau_running_loans_latest` rl
#        on rl.merchant_id = lrv.merchant_id
#        where lrv.merchant_id in (
#               SELECT
#         merchant_id
#       FROM (
#         SELECT
#       merchant_id,
#       COUNT(DISTINCT id) AS txns,
#       SUM(amount) AS tpv
#     FROM
#       bharatpe-analytics-prod.payin.transactions
#     WHERE
#       status = "SUCCESS"
#       AND DATE(payment_timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)
#     GROUP BY
#       1
#     HAVING
#       txns >=5 and tpv >= 150 )
#        )
# )
# """
# df = pandas_gbq.read_gbq(df,use_bqstorage_api=True)

In [3]:
df = """
SELECT
  merchant_id,
  repeat_loan,
  topup,
  COALESCE(Channel, 'Offline') AS Channel,
  COALESCE(running_bureau_loans,0) running_bureau_loans
FROM (
  SELECT
    DISTINCT lrv.merchant_id,
    repeat_loan,
    risk_segment,
    Channel,
    running_bureau_loans,
    CASE
      WHEN UPPER(risk_segment)='TOPUP' THEN 1
    ELSE
    0
  END
    AS topup
  FROM
    `easy_loan.lending_risk_variables` lrv
  LEFT JOIN
    bharatpe-analytics-prod.bharatpe_ml_data.merchant_channel mc
  ON
    lrv.merchant_id = mc.merchant_id
  LEFT JOIN
    `bharatpe-analytics-prod.bharatpe_ml_data.bureau_running_loans_latest` rl
  ON
    rl.merchant_id = lrv.merchant_id
  WHERE
    lrv.merchant_id IN (
    SELECT
      DISTINCT merchant_id
    FROM
      `bharatpe_analytics_data.lending_risk_variables` lrv
    WHERE
      DATE(created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 180 day) -- became eligible in last 90 days, feel free to change or comment this condition
      AND final_offer >5100
      AND merchant_id IN (
      SELECT
        merchant_id
      FROM (
        SELECT
          merchant_id,
          COUNT(DISTINCT id) AS txns,
          SUM(amount) AS tpv
        FROM
          bharatpe-analytics-prod.payin.transactions
        WHERE
          status = "SUCCESS"
          AND DATE(payment_timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
        GROUP BY
          1
        HAVING
          txns >=20
          AND tpv >= 500 ) ) ) )
"""
df = pandas_gbq.read_gbq(df,use_bqstorage_api=True)

Downloading: 100%|██████████|


In [4]:
df['existing_bureau_loan'] = np.where(df['running_bureau_loans'] >0,1,0)

In [5]:
repeat = df[df['repeat_loan'] ==1]
new = df[df['repeat_loan'] ==0]

In [6]:
repeat.head()

,merchant_id,repeat_loan,topup,Channel,running_bureau_loans,existing_bureau_loan
12,562408,1,1,Digital,3,1
68,38739970,1,1,Offline,2,1
90,37502431,1,0,Offline,7,1
99,36809449,1,1,Digital,0,0
100,37018801,1,1,Offline,3,1


In [7]:
repeat_cohort = pd.pivot_table(repeat, index='topup', columns='Channel', values='merchant_id', aggfunc='count', fill_value=0)
repeat_cohort

Channel,Digital,Offline
topup,,
0,1118,1587
1,2135,2587


In [8]:
new_cohort = pd.pivot_table(new, index='existing_bureau_loan', columns='Channel', values='merchant_id', aggfunc='count', fill_value=0)
new_cohort

Channel,Digital,Offline
existing_bureau_loan,,
0,17118,68188
1,2775,4584


In [9]:
df

,merchant_id,repeat_loan,topup,Channel,running_bureau_loans,existing_bureau_loan
0,37483938,0,0,Digital,0,0
1,2368431,0,0,Offline,0,0
2,420728,0,0,Offline,0,0
3,36608512,0,0,Offline,0,0
4,8098393,0,0,Offline,0,0
...,...,...,...,...,...,...
100087,27397047,1,1,Digital,3,1
100088,36219710,1,1,Digital,2,1
100089,35757549,1,1,Offline,6,1
100090,35026848,1,0,Offline,0,0


In [10]:
df['repeat_loan'].sum()

7427